In [1]:
import difflib
from collections import defaultdict

import h5py
import pandas as pd

# Using readlines()
import torch
import numpy as np
from Bio import SeqIO
from tqdm import tqdm

file1 = open('../data/embeddings/subCell_all_pssmFile.txt', 'r')
Lines = file1.readlines()

fasta_sequences = []
for record in tqdm(SeqIO.parse(open('../data/embeddings/subCell_all.fasta'), 'fasta')):
    fasta_sequences.append(str(record.seq))

count = 0
# Strips the newline character
skipnext = False

pssms = {}
lengths = defaultdict(list)
sequence = []
pssm = []
for i, line in tqdm(enumerate(Lines[2:])):
    if not skipnext:
        string = line.strip().split()
        if string[0] == 'Query':
            skipnext = True
            pssm_sequence = ''.join(sequence)
            pssms[pssm_sequence] = torch.stack(pssm)
            lengths[str(len(pssm_sequence))].append(pssm_sequence)
            sequence = []
            pssm = []
        else:
            if string[1] != 'X':
                sequence.append(string[1])
            pssm.append(torch.tensor(np.array(string[2:], dtype=int)))
    else:
        skipnext = False

14494it [00:00, 101356.55it/s]
7603350it [02:55, 43293.53it/s]


In [2]:
for i, fasta_sequence in tqdm(enumerate(fasta_sequences)):
    if str(len(fasta_sequence)) not in list(lengths.keys()):
        print(len(fasta_sequence))
        print(fasta_sequence)


14494it [00:00, 28522.75it/s]


2551
MVFSYACMVLQRIVVPAVLVLAALMRPVGISFVYLLMFFVSPFVPLATRRNFKGSVTAFFIILLTLSTLVLLGHITLQILAVSLTLPIYNCSFSERLLRHIGFVSFIDLQPFAIIEWLVPEVLVFATSLGSYLTVKRVASQPVGAEQLENGEVVDGQAENAQTSSQPSAADANGGDVQQATVTTPLQQQQQQLRKRVSMISQHIHFEGLVKISPLFCLATLFFAAVLRPSVPGGFYFLIFLLSGTYWATCQTLQRGFALLLRCVMVVLVLHSLSIVSYQTPWMQSHLNHTTLTARLIGLEPLIESYCSPDIRVFLYNNKLSLDSYLNPFALFFAYFALALTTKHLIKPRLVRQSTRKARTPQPLESGSSVAPSVTQRGNDMQLESMEQRSEQENTTTSILDQISYGFVSVGGFIYQNSYIFTNILMMAWSIVYHSWLTFVLLLSANVLWMIPNQRKAMMRSSPFIVLYAEALLIAQYIYGMDLNNEELPTSVPTAGINLQQIGFERPIENQMRPCVPLIVKTAFVLMFWVTSRQFFKEKRDRRRDSTLADFIAPLQITVGSAGSSYLINDGKKTSKFLKKAGDVIKNLLVRLWIWLLVLVIFLCAITGENMTGFRICYMALFLFFLLVFQSSSKAWVKIMYGFWLFLIFYAMSILILIYTYQFDKFDTYWSDYLNVSATLQKDIGLKRYQTKDLFLHLVSPTIIVILTVIQVHYFHKRFIASLQQQPLAGGSAQQKPTETTALEPAPSKRRGSAGSLRKSQGPSAEAAPGATTDFETSVRDLVRISFRKIKNKSEYIFKNFKDVFWRFLELHIMKAVYIAAFVCSVSEVCVLHIIFVGFCVLGATSRKAVQVVISRLISFIVTVIVLSKMIYQIEYLSHSQHNVVCSDNRTANNAEWIGLTKADKVTGGLMSLLRTYIIYMVIVTMHAVISLRQLQMRVKIGALNAPPTKLLFPNIIRADAEKDLVGLVKYLLNFGFYKFGIEISLIALVSTITY

In [3]:
def closest(lst, K):

     lst = np.asarray(lst, dtype=int)
     idx = (np.abs(lst - K)).argmin()
     return lst[idx]


for key ,val in lengths.items():
    if val == []:
        print(key)

dupes = []
with h5py.File('../data/embeddings/subCell_all_pssmFile.h5', 'w') as hf:
    for i, fasta_sequence in tqdm(enumerate(fasta_sequences)):
        lookup_sequences = lengths[str(len(fasta_sequence))]
        if lookup_sequences == []:
            close = closest(list(lengths.keys()), len(fasta_sequences))
            lookup_sequences = lengths[str(close)]
        if lookup_sequences == []:
            lookup_sequences = lengths['1000']
        sequence_id = difflib.get_close_matches(fasta_sequence, lookup_sequences, n=1, cutoff=0)[0]
        pssm = pssms[sequence_id]
        if fasta_sequence not in dupes:
            hf.create_dataset(fasta_sequence, data=pssms[sequence_id])
        dupes.append(fasta_sequence)

14494it [03:38, 66.47it/s]
